In [ ]:
import time
import pandas as pd
from tensorflow.keras.models import load_model
from data_preprocessing import load_dataset

In [ ]:
# Adjust the import paths as necessary
base_model = load_model('base_student_model.h5')
distilled_model = load_model('distill_student_model.h5')
teacher_model = load_model('resnet152_plant_disease.h5')

In [ ]:
base_model.summary()

In [ ]:
distilled_model.summary()

In [ ]:
teacher_model.summary()

In [ ]:
train_ds, val_ds, test_ds, class_names, num_classes = load_dataset()

In [ ]:
baseline_eval = base_model.evaluate(test_ds.map(lambda inputs, label: (inputs[1], label)), return_dict=True)
distilled_eval = distilled_model.evaluate(test_ds.map(lambda inputs, label: (inputs[1], label)), return_dict=True)
teacher_eval = teacher_model.evaluate(test_ds.map(lambda inputs, label: (inputs[0], label)), return_dict=True)

In [ ]:


# Prepare a batch of test images (1 or more)
test_images = next(iter(test_ds.take(1)))[0]  # Get images only (without labels)

# Measure teacher model inference time
start_time = time.time()
_ = teacher_model.predict(test_images[0])
teacher_time = time.time() - start_time

# Measure student model inference time
start_time = time.time()
_ = distilled_model.predict(test_images[1])
distilled_model_time = time.time() - start_time

start_time = time.time()
_ = base_model.predict(test_images[1])
base_time = time.time() - start_time


In [ ]:
# Create table data
data = [
    {
        "Model": "Teacher (ResNet152)",
        "Train Samples": 12000,
        "Epochs": 12,
        "Input Size": "224x224",
        "Accuracy": round(teacher_eval['accuracy'], 4),
        "Loss": round(teacher_eval['loss'], 4),
        "Model Size (# Parameters)": 58638120,
        "Inference Time": teacher_time
    },
    {
        "Model": "Base Mobilenetv2",
        "Train Samples": 12000,
        "Epochs": 80,
        "Input Size": "224x224",
        "Accuracy": round(baseline_eval['sparse_categorical_accuracy'], 4),
        "Loss": round(baseline_eval['loss'], 4),
        "Model Size (# Parameters)": 2595688,
        "Inference Time": base_time
    },
    {
        "Model": "Student (Distilled_Mobilenetv2)",
        "Train Samples": 12000,
        "Epochs": 80,
        "Input Size": "224x224",
        "Accuracy": round(distilled_eval['sparse_categorical_accuracy'], 4),
        "Loss": round(distilled_eval['loss'], 4),
        "Model Size (# Parameters)": 2595688,
        "Inference Time": distilled_model_time
    }
]

# Create DataFrame
df = pd.DataFrame(data)
df
